In [149]:
import networkx as nx
from pathlib import Path
from time import time
from utilities import *
from EEF import *
from WCF import *
from joblib import Parallel, delayed
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [262]:
data = 'Reddit'
graph_path = './'+data+'/'
graph_file = list(Path(graph_path).glob('*'))
list_G = Parallel(n_jobs=-1)(delayed(nx.read_gml)(g) for g in graph_file)

theta_thres_all = []
wcf_indices = []
theta_thres_all = Parallel(n_jobs=-1)(delayed(theta_thres_table)(g) for g in list_G)
wcf_indices = Parallel(n_jobs=-1)(delayed(theta_tree)(theta_thres_all[i],g) for i,g in enumerate(list_G))

In [31]:
def local_theta_core1(G, theta, k, query):
    res = nx.Graph()
    filtered_edges = [(u,v) for (u,v) in G.edges if G[u][v]['weight']>=theta]
    G_theta = G.edge_subgraph(filtered_edges)
    G_core = nx.k_core(G_theta,k)
    if query in G_core.nodes:
        res = G_theta.subgraph(nx.node_connected_component(G_core, query))
    return res


In [277]:
theta = 0.2
k=3
query = 'pics'
V_MAX = get_V_max(list_G,k)
print(V_MAX)

875


In [278]:
maxS1, C_opt1, d1 = EEF(list_G, query, theta, k, V_MAX, alpha=5)
len(C_opt1.nodes), d1

(21, 18)

In [279]:
maxS2, C_opt2, score, L_c, OptD = WCF_search_newUB(list_G, wcf_indices, query, theta, k, V_MAX, alpha=5)
len(C_opt2.nodes), OptD

(21, 18)

In [264]:
gg = list_G[4]
# aa = local_k_core(remove_theta(gg,theta, query), query,k)
aa = return_C1(gg, wcf_indices[4], query, theta, k)
len(aa.nodes)

246
154


440

In [194]:
tree = wcf_indices[4][3]
# all_v = tree['node_id'][70].get_subgraph_in_tree(tree['node_id'])[1]

In [202]:
theta_thres_all[4].loc['nba']

1     0.9
2     0.9
3     0.7
4     0.5
5     0.3
6     0.2
7     0.1
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
Name: nba, dtype: float64

In [207]:
for i in tree['node_id'].keys():
    if tree['node_id'][i].contains_v('nba'):
        print(i)

KeyError: 'node_id'

In [180]:
aa = local_k_core(remove_theta(gg,theta, query), query,k)

In [198]:
tree['node_id'][3].info()

Node: 3
verteices: ['futurology', 'science', 'theydidthemath', 'dota2', 'leagueoflegends', 'gamingcirclejerk', 'pcmasterrace', 'againstsocialjustice', 'mensrights', 'thebluepill', 'relationships', 'theredpill', 'buttcoin', 'bitcoinmarkets', 'technology', 'panichistory']
theta: 0.7
parent: 9
children: {1}


In [230]:
tree4 = theta_tree(theta_thres_all[4], gg)

you a  0.9 1
meile  0.9 1
you a  0.9 2
meile  0.9 2
you a  0.7 3
meile  0.7 3
you a  0.5 4
meile  0.5 4
you a  0.3 5
you a  0.2 6
you a  0.1 7
you a  0.0 8
you a  0.0 9
you a  0.0 10
you a  0.0 11
you a  0.0 12


In [222]:
theta_thres_all[4].loc['nba']

1     0.9
2     0.9
3     0.7
4     0.5
5     0.3
6     0.2
7     0.1
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
Name: nba, dtype: float64

In [232]:
bb = remove_theta(gg,0.7)
'nba' in bb.nodes

True

In [254]:
g = theta_thres_all[4].groupby(3)
node_v = g.get_group(0.7).index.values.tolist()
len(node_v)

36

In [256]:
sub_G = nx.subgraph(list_G[4],node_v)
temp_G = remove_theta(sub_G,0.7)
len(sub_G.nodes), len(temp_G.nodes)

(36, 16)

In [260]:
G_temp = sub_G.copy()
for (u,v) in G_temp.edges:
    if G_temp[u][v]['weight']<theta:
        G_temp.remove_edge(u,v)
len(G_temp.nodes)

36

In [1]:
G_temp.nodes

NameError: name 'G_temp' is not defined

In [10]:
nodes = ['12', '34', '45']
interval = (3,6)
score = 0.4353456

file_name = 'test.output-1-2-3_WCF'
    
f = open('./Output/'+file_name, 'w')
f.writelines('Score: ' + str(score) +'\n')
f.write('Interval: ' + str(interval) +'\n')
f.write('Nodes: ' + str(nodes))
f.close()